**Open this notebook from google drive**<br>
**Go to "Edit" -> "Notebook settings" and enable GPU.**


In [ ]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

**Connect and authorize google drive with google colab:**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

**Open our project "Galaxy Classifier" direct0ry in google drive:**

In [ ]:
# %cd /content/gdrive/My Drive/
%cd /content/gdrive/My Drive/Colab Notebooks/galaxy_classifier/
!ls

**Data for our "Galaxy Classifier" directory in google drive:**

In [ ]:
%cd /content/gdrive/My Drive/data/galaxy_data/
!ls

**Install all required libraries for our project:**

In [ ]:
# !pip install -r ./requirements.txt

In [ ]:
import os, random, shutil
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  

import tensorflow as tf
print(tf.__version__)
tf.test.gpu_device_name()

**Test if TensorFlow works with gpu for you, in output should see similar results:**
```
2.2.0
'/device:GPU:0'
```

In [ ]:
current_dir = os.getcwd()
print(current_dir)

data_path    = '/content/gdrive/My Drive/data/galaxy_data'
training_dir = os.path.join(data_path, 'training')
valid_dir    = os.path.join(data_path, 'validation')

In [ ]:
# Get the total images in the training and validation dataset, this should be around 8619 images for training, and 2875 for validation.
total_train = 0
for c in ['elliptical', 'lenticular', 'spiral']:
  total_train += len(os.listdir(os.path.join(training_dir, c)))
print('Total train:', total_train)

total_validation = 0
for c in ['elliptical', 'lenticular', 'spiral']:
  total_validation += len(os.listdir(os.path.join(valid_dir, c)))
print('Total validation:', total_validation)

In [ ]:
dir_augmented_data = os.path.join(data_path, 'preview')
try:
    ## if the preview folder does not exist, create
    os.mkdir(dir_augmented_data)
except:
    ## if the preview folder exists, then remove
    ## the contents (pictures) in the folder
    for item in os.listdir(dir_augmented_data):
        os.remove(dir_augmented_data + "/" + item)


# Resize the images to (64, 64) pixels
target_size = (150, 150)

# Set the batch size.
batch_size = 32

# Color channels
color_channels = 3


# Create the data generator for both sets, but now with image augmentation,
# where random operations are made on images like random shifts or rotationson each batch,
# stil do the rescale and set the target size.

# datagen for training set
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                    rescale            = 1./255.,
                    rotation_range     = 25,
                    width_shift_range  = 0.15,
                    height_shift_range = 0.15,
                    horizontal_flip    = True,
                    zoom_range         = 0.2)


train_generator = train_datagen.flow_from_directory(training_dir,
                                                    target_size = target_size,
                                                    batch_size  = batch_size,
                                                    shuffle     = True,
                                                    class_mode  = 'categorical')


# datagen for validation set
validation_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255.)
validation_generator = validation_datagen.flow_from_directory(valid_dir,
                                                         target_size = target_size,
                                                         batch_size  = batch_size,
                                                         shuffle     = False,
                                                         class_mode  = 'categorical')

allclasses = train_generator.class_indices    
print(allclasses)

In [ ]:
# To see what's going on this data augmentation, run the following
if(False):
  i = 0
  for batch in train_datagen.flow_from_directory(training_dir,
      target_size = target_size, batch_size  = batch_size,
      save_to_dir = dir_augmented_data, save_prefix = 'pic', save_format = 'png',
      color_mode = 'grayscale', shuffle = True, class_mode  = 'categorical'):

      i += 1
      if i > 5: # save 20 images
          break  # otherwise the generator would loop indefinitely

**The model**

In [ ]:
# Create the model, using a sequence of convolution and pooling layers,
# followed by a dropout layer, a fully connected layer, and the last softmax layer.

# Xem: https://forum.machinelearningcoban.com/t/tutorial-tinh-so-luong-parameters-trong-convolutional-neural-network/3638


model = tf.keras.models.Sequential([
    # first convolution layer, input is an 64x64 image x3 colors
    # 64 filters, with a size of (3x3) for each filter (each filter has 3x3 = 9 weights)
    # e.g: input = 28x28 pixels
    #      kernel filter = (3,3)
    #      No. of filters: 32
    #     -> Output of Conv2D layer: (28 x 28 x 32)
    tf.keras.layers.Conv2D(64, (3,3), activation='relu',
                           input_shape=(target_size[0],target_size[1], color_channels)),
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # second convolution layer
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # third convolution layer
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # fourth convolution layer
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # flatten the image pixels
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    # 512 neuron fully connected hidden layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

model.summary()  # inspect model


# Compile the model to use a categorical cross entropy loss function, and a adam optmizier.
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

**Train the model**

In [ ]:
# Train the model for 100 epochs.
EPOCHS = 100

# Train the model here
history = model.fit(train_generator,
                    epochs          = EPOCHS,
                    validation_data = validation_generator,
                    verbose         = True)

In [ ]:
import pickle
print(data_path + '/train_hist_dict.pkl')
with open(data_path + '/train_hist_dict.pkl', 'wb') as file_hist:
        pickle.dump(history.history, file_hist)

**Finish training -> plot graphs**

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))  # range for the number of epochs

plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.savefig(data_path + '/plots.png')
plt.show()

**Save the trained model**

In [ ]:
model.save(data_path + '/galaxy_convnet_rgb.h5')